In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("glue", "sst2", split = "test[:20]")
print(dataset[:5])

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

{'sentence': ['uneasy mishmash of styles and genres .', "this film 's relationship to actual tension is the same as what christmas-tree flocking in a spray can is to actual snow : a poor -- if durable -- imitation .", 'by the end of no such thing the audience , like beatrice , has a watchful affection for the monster .', 'director rob marshall went out gunning to make a great one .', 'lathan and diggs have considerable personal charm , and their screen rapport makes the old story seem new .'], 'label': [-1, -1, -1, -1, -1], 'idx': [0, 1, 2, 3, 4]}


In [3]:
import transformers 
import torch
model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B" 
pipeline = transformers.pipeline( 
"text-generation",  
model=model_id,  
max_new_tokens=2048,  
model_kwargs={"torch_dtype": torch.bfloat16},  
device_map="auto" 
) 
prompt = """Below is an instruction that describes a task,  
paired with an input that provides further context. Write a  
response that appropriately completes the request. 
Before answering, think carefully about the question and  
create a step-by-step chain of thoughts to ensure a logical  
and accurate response. 
### Instruction: 
You are an expert of consumer comment analysis. Analyze the given text from 
an online review and determine  
the sentiment polarity. Return a single number of either -1  
and 1, with -1 being negative and 1 being the positive  
sentiment.  
### Input: 
this film 's relationship to actual tension is the same as  
what christmas-tree flocking in a spray can is to actual  
snow : a poor -- if durable -- imitation . 
### Response: 
<think>""" 
response = pipeline(prompt) 
print(response[0]["generated_text"]) 

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Device set to use cuda:0


Below is an instruction that describes a task,  
paired with an input that provides further context. Write a  
response that appropriately completes the request. 
Before answering, think carefully about the question and  
create a step-by-step chain of thoughts to ensure a logical  
and accurate response. 
### Instruction: 
You are an expert of consumer comment analysis. Analyze the given text from 
an online review and determine  
the sentiment polarity. Return a single number of either -1  
and 1, with -1 being negative and 1 being the positive  
sentiment.  
### Input: 
this film 's relationship to actual tension is the same as  
what christmas-tree flocking in a spray can is to actual  
snow : a poor -- if durable -- imitation . 
### Response: 
<think>
Alright, so I need to analyze this online review and determine the sentiment polarity. The instruction says to return a single number, either -1 for negative or 1 for positive. Let's break down the text step by step.

The review says

In [4]:
from datasets import load_dataset
import transformers
import torch

# 加载 SST-2 数据集
dataset = load_dataset("glue", "sst2", split="test[:20]")  # 只取前20条样本
# 查看数据集
print(dataset)

model_id = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
# 创建生成器模型
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    max_new_tokens=2048,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

# 定义提示词模板
prompt_style = """Below is an instruction that describes a task, paired with 
an input that provides further context. Write a response that appropriately 
completes the request. 
Before answering, think carefully about the question and create a step-by-
step chain of thoughts to ensure a logical and accurate response. 
### Instruction: 
Analyze the given text from an online review and determine the sentiment 
polarity. Return a single number of either -1 and 1, with -1 being negative 
and 1 being the positive sentiment.  
### Input: 
{} 
### Response: 
<think>{}"""

correct_predictions = 0  # 用来统计正确预测数量
total_predictions = 0  # 总预测数

# 逐条处理数据集
for i, item in enumerate(dataset):
    input_text = item['sentence']
    response = pipeline(prompt_style.format(input_text, ""))
    
    # 获取生成的文本
    generated_text = response[0]["generated_text"]
    
    # 假设生成的文本包含预测的情感标签，需要从中提取该标签
    predicted_label = 1 if "1" in generated_text else -1  # 假设文本中出现1为正面情感，-1为负面情感
    
    # 输出生成的文本和真实标签
    print(f"Predicted: {predicted_label}, True label: {item['label']}")
    
    # 判断预测是否正确
    if predicted_label == item['label']:
        correct_predictions += 1
    total_predictions += 1

# 计算准确率
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy:.4f}")


Dataset({
    features: ['sentence', 'label', 'idx'],
    num_rows: 20
})


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cpu


Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Predicted: 1, True label: -1
Accuracy: 0.0000
